In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from pathlib import Path

In [2]:
from typing import Literal
from datasets import Dataset, DatasetDict, load_dataset

class YambdaDataset:
    INTERACTIONS = frozenset([
        "likes", "listens", "multi_event", "dislikes", "unlikes", "undislikes"
    ])

    def __init__(
        self,
        dataset_type: Literal["flat", "sequential"] = "flat",
        dataset_size: Literal["50m", "500m", "5b"] = "50m"
    ):
        assert dataset_type in {"flat", "sequential"}
        assert dataset_size in {"50m", "500m", "5b"}
        self.dataset_type = dataset_type
        self.dataset_size = dataset_size

    def interaction(self, event_type: Literal[
        "likes", "listens", "multi_event", "dislikes", "unlikes", "undislikes"
    ]) -> Dataset:
        assert event_type in YambdaDataset.INTERACTIONS
        return self._download(f"{self.dataset_type}/{self.dataset_size}", event_type)

    def audio_embeddings(self) -> Dataset:
        return self._download("", "embeddings")

    def album_item_mapping(self) -> Dataset:
        return self._download("", "album_item_mapping")

    def artist_item_mapping(self) -> Dataset:
        return self._download("", "artist_item_mapping")

    @staticmethod
    def _download(data_dir: str, file: str) -> Dataset:
        data = load_dataset("yandex/yambda", data_dir=data_dir, data_files=f"{file}.parquet")
        # Returns DatasetDict; extracting the only split
        assert isinstance(data, DatasetDict)
        return data["train"]


In [3]:
dataset = YambdaDataset("sequential", "50m")
data = dataset.interaction("multi_event")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'uid': [100],
 'timestamp': [[39420,
   39420,
   39625,
   40110,
   40360,
   40380,
   40640,
   40785,
   40885,
   41130,
   41545,
   41580,
   41580,
   42115,
   42115,
   42115,
   42115,
   42115,
   42115,
   42210,
   42380,
   42520,
   43425,
   43425,
   43425,
   43425,
   43425,
   43720,
   43720,
   43910,
   44335,
   44585,
   44700,
   44755,
   44755,
   44970,
   45330,
   45415,
   46535,
   46535,
   46535,
   46535,
   46535,
   46535,
   46750,
   68205,
   68205,
   68205,
   68205,
   68395,
   68600,
   82455,
   104105,
   104250,
   104430,
   104700,
   104720,
   104725,
   287910,
   287910,
   288055,
   288160,
   288290,
   288420,
   288580,
   288745,
   288865,
   289090,
   289305,
   289460,
   289565,
   289815,
   289870,
   394180,
   394180,
   394180,
   394180,
   394180,
   394530,
   1014515,
   1014665,
   1014800,
   1014920,
   1016030,
   1155305,
   1155475,
   1155575,
   1155860,
   1156590,
   1157035,
   1157035,
   1157180,

In [4]:
df = data.to_pandas()
df.head()

,uid,timestamp,item_id,is_organic,played_ratio_pct,track_length_seconds,event_type
0,100,"[39420, 39420, 39625, 40110, 40360, 40380, 406...","[8326270, 1441281, 286361, 732449, 3397170, 78...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[100.0, 100.0, 100.0, 100.0, 46.0, 100.0, 100....","[170.0, 105.0, 185.0, 240.0, 130.0, 205.0, 205...","[listen, listen, listen, listen, listen, liste..."
1,200,"[14329075, 14329075, 14329075, 14329075, 14329...","[3285270, 5253582, 9155883, 6113313, 1142869, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[9.0, 28.0, 42.0, 32.0, 0.0, 0.0, 5.0, 9.0, 10...","[170.0, 170.0, 205.0, 155.0, 215.0, 185.0, 205...","[listen, listen, listen, listen, listen, liste..."
2,300,"[54090, 54100, 54110, 54120, 54125, 54135, 541...","[618910, 8793425, 8842650, 5806319, 8495320, 9...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2.0, 4.0, 4.0, 5.0, 1.0, 5.0, nan, 7.0, 1.0, ...","[270.0, 130.0, 225.0, 210.0, 300.0, 210.0, nan...","[listen, listen, listen, listen, listen, liste..."
3,400,"[2980800, 3157520, 3417540, 3417595, 3417600, ...","[399811, 1575108, 1575108, 1575108, 9229946, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[dislike, dislike, like, unlike, dislike, like..."
4,500,"[22695440, 22695690, 22695715, 22696095, 22696...","[6417502, 6896222, 6896938, 6950641, 7981221, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, ...","[100.0, 37.0, 10.0, 37.0, 100.0, 100.0, 11.0, ...","[225.0, 210.0, 240.0, 165.0, 150.0, 245.0, 135...","[listen, listen, listen, listen, listen, liste..."


In [14]:
user = df.iloc[0]

In [23]:
user[['item_id', 'timestamp', 'event_type', 'played_ratio_pct', 'track_length_seconds']]

,0
item_id,"[8326270, 1441281, 286361, 732449, 3397170, 78..."
timestamp,"[39420, 39420, 39625, 40110, 40360, 40380, 406..."
event_type,"[listen, listen, listen, listen, listen, liste..."
played_ratio_pct,"[100.0, 100.0, 100.0, 100.0, 46.0, 100.0, 100...."
track_length_seconds,"[170.0, 105.0, 185.0, 240.0, 130.0, 205.0, 205..."


In [50]:
a = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

print(len(a[2: 7]))
print(7-2)

5
5


In [3]:
# ----------------- НАСТРОЙКИ -----------------
INPUT_PATH = 'multi_event.parquet'   # твой файл
OUTPUT_PATH = 'yambda_sessions_threshold_900.parquet'  # куда сохраним
THRESHOLD = 900          # 75 минут
BATCH_SIZE = 5           #
DROPOUT_BATCH_RATE = 0.6
CHUNK_SIZE = 2_000      # размер чанка — подбери под свою RAM (10k нормально)
# -------------------------------------------

def split_sessions_for_user(row, threshold):
    timestamps = np.array(row['timestamp'], dtype=np.int64)
    if len(timestamps) < 2:
        return [row]  # одна сессия

    diffs = np.diff(timestamps)
    split_idx = np.where(diffs > threshold)[0] + 1
    starts = np.concatenate([[0], split_idx])
    ends = np.concatenate([split_idx, [len(timestamps)]])

    sessions = []
    for i in range(len(starts)):
        start, end = starts[i], ends[i]
        length = end - start

        # Filter the length of the session
        if length < BATCH_SIZE:
            continue
        elif length < 2*BATCH_SIZE:
          if np.random.uniform(0, 1) < DROPOUT_BATCH_RATE:
            continue

        sessions.append({
            'uid': row['uid'],
            'session_idx': i,
            'session_length': length,
            'item_ids': row['item_id'][start:end],
            'played_ratio_pct': row['played_ratio_pct'][start:end] if row['played_ratio_pct'] is not None else None,
            'event_type': row['event_type'][start:end]
        })
    return sessions

# ----------------- ОСНОВНОЙ ЦИКЛ -----------------
print("Загружаем весь датасет...")
df = pd.read_parquet(INPUT_PATH, columns=['uid', 'timestamp', 'item_id', 'played_ratio_pct', 'event_type'])

all_sessions = []
for _, row in df.iterrows():
    row_dict = row.to_dict()
    sessions = split_sessions_for_user(row_dict, THRESHOLD)
    all_sessions.extend(sessions)

# Создаём DF и сохраняем
sessions_df = pd.DataFrame(all_sessions)
sessions_df.to_parquet(OUTPUT_PATH, engine='pyarrow', index=False, compression='snappy')

print(f"\nГотово! Файл: {OUTPUT_PATH}")
print(f"Количество сессий: {len(sessions_df)}")
print(sessions_df.head())


Загружаем весь датасет...

Готово! Файл: yambda_sessions_threshold_900.parquet
Количество сессий: 1663411
   uid  session_idx  session_length  \
0  100            0              22   
1  100            1              16   
2  100            2               7   
3  100            5               6   
4  100            6              15   

                                            item_ids  \
0  [8326270, 1441281, 286361, 732449, 3397170, 78...   
1  [7498961, 3630949, 2729049, 732246, 5910506, 1...   
2  [4316182, 3175575, 4747863, 6080355, 2478829, ...   
3  [7174812, 4078280, 8384633, 1524654, 2192543, ...   
4  [5699545, 8646493, 5142331, 8237502, 7688570, ...   

                                    played_ratio_pct  \
0  [100.0, 100.0, 100.0, 100.0, 46.0, 100.0, 100....   
1  [100.0, 100.0, 100.0, 100.0, 23.0, 100.0, 100....   
2    [100.0, 76.0, 100.0, 14.0, 100.0, 100.0, 100.0]   
3              [100.0, 100.0, 100.0, 0.0, 14.0, 1.0]   
4  [100.0, 100.0, 100.0, 100.0, 100.0, 100

In [5]:
sessions_df.head()

,uid,session_idx,session_length,item_ids,played_ratio_pct,event_type
0,100,0,22,"[8326270, 1441281, 286361, 732449, 3397170, 78...","[100.0, 100.0, 100.0, 100.0, 46.0, 100.0, 100....","[listen, listen, listen, listen, listen, liste..."
1,100,1,16,"[7498961, 3630949, 2729049, 732246, 5910506, 1...","[100.0, 100.0, 100.0, 100.0, 23.0, 100.0, 100....","[listen, listen, listen, listen, listen, liste..."
2,100,2,7,"[4316182, 3175575, 4747863, 6080355, 2478829, ...","[100.0, 76.0, 100.0, 14.0, 100.0, 100.0, 100.0]","[listen, listen, listen, listen, listen, liste..."
3,100,5,6,"[7174812, 4078280, 8384633, 1524654, 2192543, ...","[100.0, 100.0, 100.0, 0.0, 14.0, 1.0]","[listen, listen, listen, listen, listen, listen]"
4,100,6,15,"[5699545, 8646493, 5142331, 8237502, 7688570, ...","[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100...","[listen, listen, listen, listen, listen, liste..."
